# Compute IP/EA with XTB
Check on how the calculations are doing. What the run times are, what fractions have completed, etc.

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from moldesign.simulate.qcfractal import GeometryDataset, SolvationEnergyDataset, collect_molecular_properties, f
from qcelemental.physical_constants import constants
import pandas as pd
import numpy as np

RDKit WARNING: [14:16:37] Enabling RDKit 2019.09.3 jupyter extensions


## Get the Geometries
Gather the XTB geometries

In [3]:
dataset = GeometryDataset('Electrolyte Geometry XTB', 'xtb')

Print out the general statusqcelemental

In [4]:
dataset.coll.status()

,xtb
COMPLETE,13556
ERROR,2


Review the completed geometries

In [5]:
%%time
geoms = dataset.get_geometries()
print(f'Completed {sum(map(len, geoms.values()))} geometries of {len(geoms)} molecules')
print(f'All ionic states computed for {len(list(filter(lambda x: len(x) == 3, geoms.values())))} molecules')

Completed 13556 geometries of 4520 molecules
All ionic states computed for 4518 molecules
CPU times: user 7 s, sys: 198 ms, total: 7.2 s
Wall time: 16.1 s


## Review the Solvation Energy Calculations
Compute the solvation energies for all of the molecules

In [6]:
solv_dataset = SolvationEnergyDataset('EDW XTB Solvation Energy', 'xtb', 'xtb')

Pull down a summary

In [7]:
%%time
solv_energies = solv_dataset.get_energies()
solv_df = solv_dataset.coll.get_values()
print(f'Completed {(~solv_df.isnull()).values.sum()} solvation energies '
      f'of {len(solv_energies)} molecules')

Completed 18174 solvation energies of 1923 molecules
CPU times: user 7.75 s, sys: 98.4 ms, total: 7.85 s
Wall time: 26.9 s


## Save Redox Data to Disk
For later use

In [8]:
redox_data = collect_molecular_properties(dataset, solv_dataset)

RDKit WARNING: [14:18:49] WARNING:  Problems/mismatches: Mobile-H( Mobile-H groups: Attachment points, Number)
RDKit WARNING: [14:18:53] Cannot assign bond directions!
RDKit WARNING: [14:18:59] Cannot assign bond directions!
RDKit WARNING: [14:19:01] Cannot assign bond directions!
RDKit WARNING: [14:19:03] Cannot assign bond directions!


Add in the initial geometry

In [9]:
%%time
init_geoms = dataset.get_geometries(initial=True)
redox_data['init_geom'] = redox_data['inchi'].apply(lambda x: init_geoms.get(x, {}).get('neutral').to_string('xyz'))

CPU times: user 11.7 s, sys: 185 ms, total: 11.9 s
Wall time: 20.8 s


Save to disk

In [10]:
redox_data.to_csv('datasets/xtb_redox.csv')